<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkST0151ENSkillsNetwork20531532-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>



#### Import the required libraries we need for the lab.


In [ ]:
import piplite
await piplite.install(['numpy'],['pandas'])
await piplite.install(['seaborn'])

In [ ]:
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

#### Read the dataset in the csv file from the URL


In [ ]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv'
resp = await fetch(URL)
boston_url = io.BytesIO((await resp.arrayBuffer()).to_py())

In [ ]:
boston_df=pd.read_csv(boston_url)

#### Add your code below following the instructions given in the course to complete the peer graded assignment


In [ ]:
boston_df.describe()

In [ ]:
ax = sns.boxplot(y='MEDV', data=boston_df)

ax.set_title("Median Value of Homes in $1,000")

median = boston_df['MEDV'].median()
first_quartile = boston_df['MEDV'].quantile(0.25)
third_quartile = boston_df['MEDV'].quantile(0.75)

ax.text(x=0.05, y=first_quartile, s="25%", verticalalignment='center', horizontalalignment='right', color='blue')
ax.text(x=0.05, y=median, s="50%", verticalalignment='center', horizontalalignment='right', color='blue')
ax.text(x=0.05, y=third_quartile, s="75%", verticalalignment='center', horizontalalignment='right', color='blue')

In [ ]:
ax2 = sns.countplot(x='CHAS', data=boston_df)

ax2.set_title("Number of Homes Along the Charles River")
ax2.set_xlabel("Charles River Proximity")
ax2.set_ylabel("Number of Homes")
ax2.set_xticklabels(['Not Along River', 'Along River'])

In [ ]:
boston_df['age_group'] = pd.cut(boston_df['AGE'], 
                                bins=[0, 35, 70, 100], 
                                labels=['35% or less', 'Between 35% and 70%', '70% or more'],
                                include_lowest=True)

ax3 = sns.boxplot(x="MEDV", hue="age_group", data=boston_df, legend = False)

ax3.set_title("Median Value of Homes by Older Home Proportion Group")
ax3.set_xlabel("Median Value of Homes in $1,000s")
ax3.set_ylabel("Age Group")

**Conclusion** we can see older cohorts have Lower value homes, EXCEPTING OUTLIERS

In [ ]:
ax4 = sns.scatterplot(x = "INDUS", y = "NOX", data = boston_df)
ax4.set_xlabel("Proportion of non-retail business acres per town.")
ax4.set_ylabel("Nitric Oxide Concentration (PPM)")
pyplot.show()

**Conclusion** we can industrial real estate is more prevalent in areas with higher concentration of NOX

In [ ]:
ax5 = sns.histplot(data=boston_df, x="PTRATIO", bins=10, kde=True, color='green')
ax5.set_xlabel("Pupil-Teacher Ratio")
ax5.set_ylabel("# Homes")

**Conclusion** We can see the vast majority of homes have a teacher:pupil ratio of 18+, with 20 being the mode

**Question 1:** Is there a significant difference in median value of houses bounded by the Charles river or not?

In [ ]:
##Approach to Answer: T-Test for Independent Samples

##H0: Null Hypothesis: There is no significant difference in the median value of houses bound by the Charles River
##Ha: Alternative Hypothesis: There is a significant difference in the median value

##Independent Variable: Bound by Charles River; two Groups: Bound by River and Not Bound by River
##Dependent & Continuous Variable: Median Home Value in $1000

##Run the T-Test
scipy.stats.ttest_ind(boston_df[boston_df['CHAS'] == 1]['MEDV'],
                   boston_df[boston_df['CHAS'] == 0]['MEDV'], equal_var = True)

**Conclusion** Since the P-Value at 7.39e-05 is less than .05, we reject the null hypothesis that the there is no diference between mean median value of homes bound by the charles river vs. those not bound

**Question 2:** Is there a difference in Median values of houses (MEDV) for each proportion of owner occupied units built prior to 1940 (AGE)?

In [ ]:
##Approach to Answer: ANOVA

##H0: Null Hypothesis: There is no significant difference in the median values of homes across three home age cohorts
##Ha: Alternative Hypothesis: There is a significant difference in the median values by at least one cohort

##Groups: 35 % or less , Between 35% and 70%, 70% 
##Dependent & Continuous Variable: Median Home Value in $1000

##Group data into three age categories

thirtyfive_lower = boston_df[boston_df['AGE'] <= 35]['MEDV']
thirtyfive_seventy = boston_df[(boston_df['AGE'] > 35) & (boston_df['AGE'] <= 70)]['MEDV']
seventy_more = boston_df[boston_df['AGE'] > 70]['MEDV']

##Run the ANOVA
f_statistic, p_value = scipy.stats.f_oneway(thirtyfive_lower, thirtyfive_seventy, seventy_more)
print("F_Statistic: {0}, P-Value: {1}".format(f_statistic,p_value))

**Conclusion** because the P-Value is lower than .05, we reject the null hypothesis. Age does play a significant role in median home value.

**Question 3:** Can we conclude that there is no relationship between Nitric oxide concentrations and proportion of non-retail business acres per town? 

In [ ]:
##Approach to Answer: Pearson Correlation

##H0: Null Hypothesis: There is no significant correlation between Nitric Oxide Concentration and Proportion of Non-Retail Business Acre per Town
##Ha: Alternative Hypothesis: There is correlation between NOX levels and Industrialized acreage

##Run the Pearson Test
ax6 = sns.scatterplot(x="NOX", y="INDUS", data=boston_df)
ax6.set_xlabel("Nitric Oxide Concentration Levels")
ax6.set_ylabel("% Industrial Acreage")
scipy.stats.pearsonr(boston_df['NOX'], boston_df['INDUS'])

**Conclusion** because the P-Value is lower than .05, there is correlation between Nitric Oxide Levels and Industrial Acreage

**Question 4:** What is the impact of an additional weighted distance to the five Boston employment centres on the median value of owner occupied homes?

In [ ]:
##Approach to Answer: Regression Analysis

##H0: Null Hypothesis: There is no significant correlation between weighted distance to the five Boston employment Centers and median home value.
##Ha: Alternative Hypothesis: There is correlation between Work distance and median home value.

##Run the Regression Test
## X is the input variables (or independent variables)
X2 = boston_df['DIS']
## y is the target/dependent variable
y2 = boston_df['MEDV']
## add an intercept (beta_0) to our model
X2 = sm.add_constant(X2) 

model = sm.OLS(y2, X2).fit()
predictions = model.predict(X2)

# Print out the statistics
model.summary()

In [ ]:
**Conclusion:** because the P-Value is lower than .05, we can determine that there is correlation between distance to the five boston centers and the median home value.